In [14]:
import pandas as pd
import numpy as np

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
        

data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/test.csv', header=0,usecols=test_cols,parse_dates=["click_time"], dtype=dtypes)
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train_sample.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)

print('len read:',len(data))


data['hour'] = data["click_time"].dt.hour.astype('uint8')
data['day'] = data["click_time"].dt.day.astype('uint8')

print('unique days in sample:' ,data['day'].unique())
print('record counts in days in sample:', data[['day','channel']].groupby(['day']).count())

len read: 184903890
unique days in sample: [6 7 8 9]
record counts in days in sample:       channel
day          
6     9308568
7    59633310
8    62945075
9    53016937


In [ ]:
#print(data[1000000:1000003])
data['id']=data.index
data1 = data.set_index('click_time').loc['2017-11-08 04:00:00':'2017-11-08 15:00:00']

print(data1['id'].min())
print(data1['id'].max())
print(len(data1))

82259195
118735619
36476425


In [32]:
data1 = data.set_index('click_time').loc['2017-11-09 04:00:00':'2017-11-09 15:00:00']

print(data1['id'].min())
print(data1['id'].max())
print(len(data1))

144708152
181878211
37170060


In [22]:
# calculate combined cvr to test hist fts generation

#print('app device cvr:', data[['app','device', 'is_attributed']].groupby(['app','device']).mean().to_string())

os_count = data[['os', 'is_attributed']].groupby(['os']).count()

os_non_attr_count = (1 - data[['os', 'is_attributed']].groupby(['os']).max()) * os_count

print('app device non attri count:', os_non_attr_count.to_string())

#print('app device  attri count:', data[['app','device', 'is_attributed']].groupby(['app','device']).sum().to_string())

#attributed_data = data.query('is_attributed == 1')

#print('len of attributed:',len(attributed_data))

app device non attri count:      is_attributed
os                
0                0
1                0
2                0
3                0
4                0
5                0
6                0
7                0
8                0
9                0
10               0
11               0
12               0
13               0
14               0
15               0
16               0
17               0
18               0
19               0
20               0
21               0
22               0
23               0
24               0
25               0
26               0
27               0
28               0
29               0
30               0
31               0
32               0
34               0
35               0
36               0
37               0
38               0
39           94800
40               0
41               0
42               0
43               0
44               0
45               0
46               0
47               0
48               0
49               0
50 

In [24]:

data= data.merge(os_non_attr_count.reset_index(), on=['os'], how='left')


print(data)

               ip  app  device  os  channel          click_time  \
0           83230    3       1  13      379 2017-11-06 14:32:21   
1           17357    3       1  19      379 2017-11-06 14:33:34   
2           35810    3       1  13      379 2017-11-06 14:34:12   
3           45745   14       1  13      478 2017-11-06 14:34:52   
4          161007    3       1  13      379 2017-11-06 14:35:08   
5           18787    3       1  16      379 2017-11-06 14:36:26   
6          103022    3       1  23      379 2017-11-06 14:37:44   
7          114221    3       1  19      379 2017-11-06 14:37:59   
8          165970    3       1  13      379 2017-11-06 14:38:10   
9           74544   64       1  22      459 2017-11-06 14:38:23   
10         172522    3       1  25      379 2017-11-06 14:38:27   
11         105861    3       1  13      379 2017-11-06 14:38:51   
12         210962    3       1  19      379 2017-11-06 14:39:29   
13         124979    3       1  18      379 2017-11-06 14:40:1

In [29]:
xxx = pd.DataFrame(data['is_attributed_y'], dtype='float64')
xxx.to_csv('validation.csv.bz2', index=False, compression='bz2')

In [30]:

log_group = 100000
def rate_calculation(x):
    """Calculate the attributed rate. Scale by confidence"""
    rate = x.sum() / float(x.count())
    conf = np.min([1, np.log(x.count()) / log_group])
    return rate * conf

yyy = data[['app','device', 'is_attributed_x']].groupby(['app','device'])[['is_attributed_x']].apply(rate_calculation).reset_index()
data= data.merge(yyy, on=['app','device'], how='left')

xxx = pd.DataFrame(data['is_attributed_x_y'])
xxx.to_csv('validation1.csv.bz2', index=False, compression='bz2')

In [ ]:
print('sampling data')
#data = data.set_index('ip').loc[lambda x: (x.index + 401) % 10 == 0].reset_index()


data.drop('click_time', inplace=True, axis=1)

In [25]:
data0['hour'] = data0["click_time"].dt.hour.astype('uint8')
data0['day'] = data0["click_time"].dt.day.astype('uint8')

data0.drop('click_time', inplace=True, axis=1)

In [15]:
#data.drop('click_time', inplace=True, axis=1)

print('added hour and day')



added hour and day


In [26]:
#print(data.describe())

data0 = data0.query('hour in [{}]'.format(','.join(map(lambda x: str(x), set(data['hour'].unique())))))
print(data0['hour'].unique())

hour_set = set(data['hour'].unique())
hour_set0 = set(data0['hour'].unique())


[14 15  4  5  6  9 10 11 13]


In [27]:
osset = set(data['os'].unique())
osset0 = set(data0['os'].unique())

print(len(osset & osset0))
print(len(osset0))

315
573


In [28]:
chset = set(data['app'].unique())
chset0 = set(data0['app'].unique())

print(len(chset & chset0))
print(len(chset0))

384
610
